In [2]:
from langchain_community.vectorstores import Chroma
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt import ToolNode
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from datetime import datetime
import os
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

load_dotenv()
GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"]
persist_directory = 'docs/chroma/'

In [3]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            passenger_id = configuration.get("passenger_id", None)
            state = {**state, "user_info": passenger_id}
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}
    
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_tokens=250,
    temperature=0.5)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

vectorstore = Chroma(persist_directory=persist_directory,
                              embedding_function=embeddings)

content_retriever = vectorstore.as_retriever()

In [4]:
@tool("pdf_info_vdb_tool")
def pdf_info_vdb_tool(query:str,config: RunnableConfig) -> str:
  """
  Search for information regarding failures of devices and recommended guidelines in various scenarios.

  Args:
      query (str): user query which can be the question or any request related to the device failure details.

  Returns:
      str: retrieved relevant documents for the user query which is formatted into string datatype
  """
  retrieved_docs = content_retriever.invoke(query)

  retrieved_docs_str = "\n".join([doc.page_content for doc in retrieved_docs])
  return retrieved_docs_str

In [5]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a QA assistant for 'SenzMate AIoT Intelligence."
            " Use the provided tools to search for necessary details for device failures and solutions for those device failures "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now())

part_1_tools = [pdf_info_vdb_tool]
part_1_assistant_runnable = primary_assistant_prompt | llm.bind_tools(part_1_tools)

In [6]:
builder = StateGraph(State)


# Define nodes: these do the work
builder.add_node("assistant", Assistant(part_1_assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(part_1_tools))
# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
part_1_graph = builder.compile(checkpointer=memory)

In [7]:
config = {
    "configurable": {
        "thread_id": "th_002"
    }
}

In [8]:
question = "Can you provide me reason for temperature sensor malfunction?"

In [9]:
ans = part_1_graph.invoke(
    {"messages": ("user", question)}, config
)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [10]:
answer = ans['messages'][-1].content

"The temperature sensor could be malfunctioning due to several reasons:\n\n* **Chip failure:** The sensor's internal chip might be damaged.\n* **Sensor wire damage:** The wire connecting the sensor to the main board could be broken or damaged.\n* **Connection issue with the main board:** The connection between the sensor and the main board might be loose or faulty. \n\nDo you have any more information about the specific device or the symptoms you're observing? This would help me narrow down the potential causes and suggest more specific solutions. \n"

In [3]:
from chat_agent.qa_agent import get_answer

In [4]:
answer = get_answer("Can you provide me reason for temperature sensor malfunction?")
print(answer)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


A temperature sensor malfunction can happen for a few reasons:

* **Chip Failure:** The sensor's internal chip might be damaged, leading to inaccurate readings or complete failure.
* **Sensor Wire Damage:** A broken or damaged wire connecting the sensor to the main board can interrupt the signal, causing incorrect readings.
* **Connection Issue:** A loose or faulty connection between the sensor and the main board can also disrupt the signal flow.

If you're experiencing a temperature sensor malfunction, try checking the sensor wire for any damage and ensure a secure connection to the main board. If these steps don't fix the issue, the sensor chip might be faulty and need replacement. 

